In [ ]:
# !pip3 install -U keras-tuner
# !pip3 install tensorflow
# !pip3 install imblearn

In [6]:
from kerastuner.tuners import RandomSearch

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import keras as k
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix
import json
from sklearn.metrics import f1_score

In [7]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.combine import SMOTEENN
from collections import Counter
from sklearn.metrics import f1_score

In [8]:
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)

In [9]:
dataset_url = 'https://www.openml.org/data/get_csv/4965303/flare.arff' 
dataset = np.genfromtxt(dataset_url, delimiter=',', skip_header=1)

x = dataset[:,:-4]
x_size = x.shape[1]
y = dataset[:,-1]

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([("columns", OneHotEncoder(), [0,1])], remainder = 'passthrough')
x = ct.fit_transform(x)
x[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 3., 1., 3., 1., 1.])

In [11]:
x.shape

(1066, 17)

In [12]:
# Contrucción de los conjuntos de entrenamiento y validación

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, test_size=0.2) 

In [13]:
smoteen = SMOTEENN()
X_ajust_train, y_ajust_train = smoteen.fit_resample(x_train, y_train)
print(Counter(y_ajust_train))

Counter({0.0: 417, 1.0: 417})


In [14]:
metrics = [keras.metrics.BinaryAccuracy(name='ACC')]

In [22]:
def work(build_model, max_trials=50, epochs=75):
    !rm -rf untitled_project/
    tuner = RandomSearch(build_model, objective='val_ACC', max_trials=max_trials, executions_per_trial=3)
    tuner.search_space_summary()
    tuner.search(x_train, y_train, epochs=epochs, validation_data=(x_val, y_val), verbose=0)
    tuner.results_summary()
    return tuner

In [23]:
def find_F1_score(tuner):
    best_nn = tuner.get_best_models()
    best_model = None
    best_score1 = 0
    best_score2 = 0
    for model in best_nn:
        y_pred = model.predict(x_val)
        y_pred = y_pred > 0.5
        f1_score1 = f1_score(y_val, y_pred, average='binary')
        f1_score2 = f1_score(y_val, y_pred, average='weighted')
        if f1_score1 > best_score1:
            best_model = model
            best_score1 = f1_score1
            best_score2 = f1_score2
        elif f1_score1 == best_score1:
            if f1_score2 > best_score2:
                best_model = model
                best_score2 = f1_score2
            


    y_pred = best_model.predict(x_val)
    y_pred = y_pred > 0.5
    print(confusion_matrix(y_val, y_pred))
    print(classification_report(y_val, y_pred, zero_division=0))
    config = json.dumps(best_model.get_config(), indent=2)
    print(config)

### Loss = binary_crossentropy

In [25]:
def build_model_mse_1(hp):
    nn = Sequential()
    nn.add(Dense(units=hp.Int('units', 5, 40), activation=hp.Choice('dense_activation1', values= ['relu', "tanh", 'sigmoid'])))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[0.01, 0.003, 0.001, 0.0003, 0.0001])), loss="mse", metrics=metrics)
    return nn

tuner = work(build_model_mse_1, max_trials=1000)
find_F1_score(tuner)

Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 40, 'step': 1, 'sampling': None}
dense_activation1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.003, 0.001, 0.0003, 0.0001], 'ordered': True}


'rm' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


INFO:tensorflow:Oracle triggered exit
Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units: 6
dense_activation1: relu
learning_rate: 0.001
Score: 0.8395638664563497
Trial summary
Hyperparameters:
units: 14
dense_activation1: relu
learning_rate: 0.003
Score: 0.836448609828949
Trial summary
Hyperparameters:
units: 38
dense_activation1: relu
learning_rate: 0.0003
Score: 0.8333333333333334
Trial summary
Hyperparameters:
units: 16
dense_activation1: relu
learning_rate: 0.0001
Score: 0.8317757050196329
Trial summary
Hyperparameters:
units: 29
dense_activation1: tanh
learning_rate: 0.01
Score: 0.8302180568377177
[[174   3]
 [ 29   8]]
              precision    recall  f1-score   support

         0.0       0.86      0.98      0.92       177
         1.0       0.73      0.22      0.33        37

    accuracy                           0.85       214
   macro avg       0.79      0.60      0.62       214
weighted avg       0.83      0.85      

In [ ]:
def build_model_mse_2(hp):
    nn = Sequential()
    nn.add(Dense(units=hp.Int('units_1', 5, 40), activation=hp.Choice(f"activation_1", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int('units_2', 4, 20), activation=hp.Choice(f"activation_2", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[0.01, 0.003, 0.001, 0.0003, 0.0001])), loss="mse", metrics=metrics)
    return nn

tuner = work(build_model_mse_2, max_trials=1000)
find_F1_score(tuner)

In [ ]:
def build_model_mse_3(hp):
    nn = Sequential()
    nn.add(Dense(units=hp.Int('units_1', 5, 40), activation=hp.Choice(f"activation_1", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int('units_2', 4, 20), activation=hp.Choice(f"activation_2", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int('units_3', 3, 10), activation=hp.Choice(f"activation_3", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[0.01, 0.003, 0.001, 0.0003, 0.0001])), loss="mse", metrics=metrics)
    return nn

tuner = work(build_model_mse_3, max_trials=500)
find_F1_score(tuner)

In [ ]:
def build_model_mse_4(hp):
    nn = Sequential()
    nn.add(Dense(units=hp.Int('units_1', 5, 40), activation=hp.Choice(f"activation_1", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int('units_2', 4, 20), activation=hp.Choice(f"activation_2", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int('units_3', 3, 10), activation=hp.Choice(f"activation_3", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int('units_4', 2, 5), activation=hp.Choice(f"activation_4", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[0.01, 0.003, 0.001, 0.0003, 0.0001)), loss="mse", metrics=metrics)
    return nn

tuner = work(build_model_mse_4, max_trials=500)
find_F1_score(tuner)